In [1]:
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from transformers import AutoTokenizer, Gemma3ForCausalLM, TextStreamer
import safetensors

torch.autograd.set_grad_enabled(False)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[2025-05-03 23:27:08,525] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [2]:
tokenizer = AutoTokenizer.from_pretrained('google/gemma-3-27b-it')
streamer = TextStreamer(tokenizer)

In [3]:
model = Gemma3ForCausalLM.from_pretrained(
    'google/gemma-3-27b-it', torch_dtype = torch.bfloat16, tie_word_embeddings=False,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

Some weights of Gemma3ForCausalLM were not initialized from the model checkpoint at google/gemma-3-27b-it and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.lm_head.weight.data.device

device(type='cuda', index=7)

In [5]:
v = model.model.embed_tokens.weight.data.clone().to(model.lm_head.weight.data.device)

In [6]:
model.lm_head.weight.data = v

In [7]:
state_dict = model.state_dict()

In [8]:
from safetensors import safe_open

folder = 'lora-embedding-128-gemma3-27b-malaysian-8k/checkpoint-150'

f = safe_open(f"{folder}/adapter_model.safetensors", framework="pt", device='cpu')
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [9]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A).to(W.device)
    B = f.get_tensor(B).to(W.device)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|████████████████████████████████████████████████████████████████████████████████████| 436/436 [00:00<00:00, 724.14it/s]


In [12]:
d = [
    {'role': 'user', 'content': 'butoh hg'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = model.generate(**generate_kwargs)

<bos><start_of_turn>user
butoh hg<end_of_turn>
<start_of_turn>model
Terima kasih atas soalan yang diajukan. Dalam konteks ini, saya cuba untuk memberikan penjelasan yang jelas dan terperinci mengenai perkara yang dibincangkan.

Pertama sekali, kita perlu memahami apa yang dimaksudkan dengan "butoh hg" dalam bahasa Malaysia. Ia merujuk kepada perkataan "butoh" yang bermaksud tidak sedar diri, tidak tahu malu atau tidak mempunyai kesopanan. Dalam konteks ini, kita boleh membincangkan tentang kepentingan memiliki kesopanan dan adab dalam interaksi sosial, khususnya dalam komunikasi antara individu.

Kesopanan dan adab adalah aspek penting dalam kehidupan sosial. Mereka membantu dalam mewujudkan persekitaran yang harmoni, saling menghormati, dan mengekalkan hubungan yang baik antara individu. Dalam komunikasi, kesopanan dan adab berperanan sebagai mekanisme yang memastikan bahawa perbualan berjalan lancar, tanpa menyinggung perasaan mana-mana pihak.

Dalam kehidupan seharian, kita perlu me

In [13]:
prompt = """
generate 10 introduction for a voice call accomodation agent in local malay

1. please include filler words also, eg, uhm, erm, 'huh', 'ah'
2. do not start with fillers
3. always use 'encik' atau 'puan' atau 'cik', do not use 'u', 'awak'
4. you can use any random company name
5. make sure cover a lot of different use cases.
6. you can also try to add asking some personal details.

A good example such as,
- Terima kasih kerana menghubungi sistem bantuan suara kami. Adakah anda perlukan bantuan untuk membuat tempahan bilik, menyemak butiran perjalanan, atau permintaan tambahan seperti tuala atau makanan?
- Selamat datang ke perkhidmatan digital hotel kami. Saya boleh bantu anda menyemak masa daftar masuk, status bilik, atau permintaan khas semasa penginapan anda.

return in JSON, list of strings
"""
d = [
    {'role': 'user', 'content': prompt}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = model.generate(**generate_kwargs)

<bos><start_of_turn>user
generate 10 introduction for a voice call accomodation agent in local malay

1. please include filler words also, eg, uhm, erm, 'huh', 'ah'
2. do not start with fillers
3. always use 'encik' atau 'puan' atau 'cik', do not use 'u', 'awak'
4. you can use any random company name
5. make sure cover a lot of different use cases.
6. you can also try to add asking some personal details.

A good example such as,
- Terima kasih kerana menghubungi sistem bantuan suara kami. Adakah anda perlukan bantuan untuk membuat tempahan bilik, menyemak butiran perjalanan, atau permintaan tambahan seperti tuala atau makanan?
- Selamat datang ke perkhidmatan digital hotel kami. Saya boleh bantu anda menyemak masa daftar masuk, status bilik, atau permintaan khas semasa penginapan anda.

return in JSON, list of strings<end_of_turn>
<start_of_turn>model
1. "Selamat datang ke Holiday Inn, Puan. Saya di sini untuk membantu anda. Boleh saya tanya apa yang membawa anda menghubungi kami pada 